### Chatbot History implemntation with sesson id details

In [2]:
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
load_dotenv()
groq_api_key=os.getenv("GROQ_API_KEY")


In [3]:
from langchain_groq import ChatGroq
model = ChatGroq(model="groq/compound",groq_api_key=groq_api_key)
model

ChatGroq(profile={}, client=<groq.resources.chat.completions.Completions object at 0x7f6550ce8c20>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7f6550c633e0>, model_name='groq/compound', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="hi i am maniram and i am software engineer")])

AIMessage(content='Hi Maniram! Nice to meet you. How can I help you today?', additional_kwargs={'reasoning_content': '<Think>\n\n</Think>'}, response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 250, 'total_tokens': 328, 'completion_time': 0.166841, 'completion_tokens_details': None, 'prompt_time': 0.009882, 'prompt_tokens_details': None, 'queue_time': 0.104629, 'total_time': 0.176723}, 'model_name': 'groq/compound', 'system_fingerprint': None, 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b984b-4735-7732-80a9-d306a150bf5e-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 250, 'output_tokens': 78, 'total_tokens': 328})

In [5]:
from langchain_core.messages import AIMessage
model.invoke([
    HumanMessage(content="hi i am maniram and i am software engineer"),
    AIMessage(content="Hello maniram, nice to meet you, how can I help you ?"),
    HumanMessage(content="what's my name and who I am ?"),
    
])

AIMessage(content='Your name is **Maniram**, and you mentioned that you’re a **software engineer**. Let me know if there’s anything specific you’d like to discuss or need help with!', additional_kwargs={'reasoning_content': '<Think>\n\n</Think>'}, response_metadata={'token_usage': {'completion_tokens': 104, 'prompt_tokens': 318, 'total_tokens': 422, 'completion_time': 0.230877, 'completion_tokens_details': None, 'prompt_time': 0.013332, 'prompt_tokens_details': None, 'queue_time': 1.067916, 'total_time': 0.244208}, 'model_name': 'groq/compound', 'system_fingerprint': None, 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b984b-6ed5-74f0-91eb-9d0adfaeb918-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 318, 'output_tokens': 104, 'total_tokens': 422})

In [9]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str) -> BaseChatMessageHistory :
    if session_id not in store :
        store[session_id]=ChatMessageHistory()
    return store[session_id]
with_message_history=RunnableWithMessageHistory(model,get_session_history)


In [10]:
config={"configurable":{"session_id":"chat1"}}

In [11]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , my name is Maniram an I am AI Engineer")],
    config
)
response.content

'Hello Maniram! Nice to meet you. How can I assist you today?'

In [18]:
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="What is my name?" )],
    config1
)
response.content

'I’m sorry, but I don’t have any information about your name. The conversation so far hasn’t included any details that would let me infer or retrieve it, so I’m unable to answer that question. If you’d like to share your name or give me more context, I’ll be happy to address any follow‑up questions you have.'

# PromptTemplate

In [20]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate(
    [
        ("system","you are a helpful assistant. Answer all questions to the best of your ability.",),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain=prompt|model

In [21]:
chain.invoke({"messages":[HumanMessage(content="My name is Krish")]})

AIMessage(content='Nice to meet you, Krish! How can I help you today?', additional_kwargs={'reasoning_content': '<Think>\n\n</Think>'}, response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 278, 'total_tokens': 354, 'completion_time': 0.163408, 'completion_tokens_details': None, 'prompt_time': 0.011151, 'prompt_tokens_details': None, 'queue_time': 0.097522, 'total_time': 0.174559}, 'model_name': 'groq/compound', 'system_fingerprint': None, 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b985b-a6d2-7261-97d9-e42473ccc0a8-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 278, 'output_tokens': 76, 'total_tokens': 354})

In [22]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [23]:
config={"configurable":{"session_id":"chat3"}}
response=with_message_history.invoke([
    HumanMessage(content="Hi, My name is Krish")
],config=config)
response.content


'Hi Krish! Nice to meet you. How can I help you today?'

In [24]:
prompt=ChatPromptTemplate(
    [
        ("system","you are a helpful assistant. Answer all questions to the best of your ability in {language}.",),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain=prompt|model

In [25]:
response=chain.invoke(
    {"messages":[HumanMessage(
        content="Hi My name is Krish"
    )],"language":"hindi"}
)
response.content

'नमस्ते\u202fKrish! आपका स्वागत है। मैं यहाँ आपकी मदद के लिए हूँ—आप मुझसे कोई भी सवाल पूछ सकते हैं या जिस विषय पर चर्चा करना चाहते हैं, बता सकते हैं। मैं पूरी कोशिश करूँगा कि आपके सभी प्रश्नों के उत्तर हिंदी में दूँ। कृपया बताइए, मैं आज आपकी कैसे सहायता कर सकता हूँ?'

In [26]:
with_message_history=RunnableWithMessageHistory(
    chain,get_session_history,
    input_messages_key="messages"
)


In [27]:
config={"configurable":{"session_id":"chat4"}}
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="Hi,my name is Krish")],"language":"Hindi"},
    config=config
)
response.content

'नमस्ते\u202fKrish! आपका स्वागत है। मैं यहाँ आपकी मदद के लिए तैयार हूँ—चाहे आपको कोई जानकारी चाहिए, कोई समस्या हल करनी हो, या बस बातचीत करनी हो। बताइए, मैं आज आपकी किस प्रकार सहायता कर सकता हूँ?'